In [1]:
!pip install tweepy

    100% |████████████████████████████████| 286kB 11.8MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 15.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for PySocks ... done
  Stored in directory: /Users/atakanokan/Library/Caches/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built PySocks
moviepy 0.2.3.2 has requirement decorator==4.0.11, but you'll have decorator 4.3.0 which is incompatible.


In [2]:
!pip install textblob

    100% |████████████████████████████████| 634kB 10.8MB/s ta 0:00:01
moviepy 0.2.3.2 has requirement decorator==4.0.11, but you'll have decorator 4.3.0 which is incompatible.


In [3]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to
[nltk_data]     /Users/atakanokan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/atakanokan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/atakanokan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/atakanokan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/atakanokan/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/atakanokan/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


Authentication

* register an App through twitter account
* https://apps.twitter.com/
* https://apps.twitter.com/app/15123027
* ‘Keys and Access Tokens’ tab
*  ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’

In [14]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
 
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = '0TJKgrI0N6HF53BuS9y22YnCD'
        consumer_secret = 'cJkg6rOnc5WSueoiFuSyVuR6hFxe880zQfU6md9dQAdt05wyvi'
        access_token = '987408772804104193-S3THYfy7YZqJjEDvulbjaGtbUZSZwyu'
        access_token_secret = 'UhRcPGGOupKZrlpE3Px23IeoDeidStGyefcN9hUPGBZqU'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
        
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
            print("Number of tweets from API: " + str(len(fetched_tweets)))
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
 

def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'Hubble lens', count = 2000)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print(len(tweets))
    print(len(ntweets))
    print(len(ptweets))
    print("Neutral tweets percentage: {} % ".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
 
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
 
if __name__ == "__main__":
    # calling main function
    main()

Number of tweets from API: 43
Positive tweets percentage: 71.42857142857143 %
Negative tweets percentage: 4.761904761904762 %
42
2
30
Neutral tweets percentage: 23.80952380952381 % 


Positive tweets:
HUBBLE | The More Affordable Daily Contact Lens. https://t.co/VHnXm2YgLN
HUBBLE | The More Affordable Daily Contact Lens. https://t.co/ikJQJs4FlY
@cstross The "Stars Are Right" conjunction caused a huge beam of occult energy to be fired from Yuggoth in our dire… https://t.co/uWAMjakJKT
@CREWcrew Hahahaha...he REFLECTS the criminality of the administration as perfectly as Hubble's lens.
HUBBLE | The More Affordable Daily Contact Lens 😍
HUBBLE | The More Affordable Daily Contact Lens. https://t.co/FjJ1bXknDh
Hubble peers through cosmic lens to capture most distant star ever seen (via @NewsfusionApps #NASA News)
https://t.co/N80mYEqOgC
@AbsoluteTruthMG First off, Hubble is facing away from the earth. It's lens is facing into space son of course Hubb… https://t.co/wL8C8Ze4dS
RT @ESAesdc: ESA 